<a href="https://colab.research.google.com/github/samantharudra/policyengine-us/blob/samantharudra%2Fissue5223/Dependent_Exemption_Reform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PolicyEngine US

Use this notebook to run the PolicyEngine US microsimulation model for custom reforms or outputs.

It's recommended to use the snippet generation features in the PolicyEngine web app for getting started.

In [1]:
%%capture
!pip install policyengine-us

In [ ]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
import pandas as pd

# Simulate reform, set dependent exemptions = 0

reform = Reform.from_dict({
  "gov.states.al.tax.income.exemptions.dependent[0].amount": {
    "2023-01-01.2100-12-31": 0
  },
  "gov.states.al.tax.income.exemptions.dependent[1].amount": {
    "2023-01-01.2100-12-31": 0
  },
  "gov.states.al.tax.income.exemptions.dependent[2].amount": {
    "2023-01-01.2100-12-31": 0
  },
  "gov.states.ca.tax.income.exemptions.dependent_amount": {
    "2024-01-01.2100-12-31": 0
  },
  "gov.states.il.tax.income.exemption.dependent": {
    "2024-01-01.2100-12-31": 0
  },
  "gov.states.in.tax.income.exemptions.additional.amount": {
    "2024-01-01.2100-12-31": 0
  },
  "gov.states.ks.tax.income.exemptions.by_filing_status.dependent": {
    "2024-01-01.2100-12-31": 0
  },
  "gov.states.la.tax.income.exemptions.dependent": {
    "2024-01-01.2100-12-31": 0
  },
  "gov.states.mn.tax.income.exemptions.amount": {
    "2024-01-01.2100-12-31": 0
  },
  "gov.states.mo.tax.income.minimum_taxable_income": {
    "2024-01-01.2100-12-31": 0
  },
  "gov.states.nj.tax.income.exemptions.dependents.amount": {
    "2024-01-01.2100-12-31": 0
  },
  "gov.states.ny.tax.income.exemptions.dependent": {
    "2024-01-01.2100-12-31": 0
  },
  "gov.states.ga.tax.income.exemptions.dependent": {
    "2023-01-01.2100-12-31": 0
  },
  "gov.states.me.tax.income.credits.dependent_exemption.amount": {
    "2023-01-01.2100-12-31": 0
  },
   "gov.states.ia.tax.income.credits.exemption.dependent": {
    "2023-01-01.2100-12-31": 0
  },
  "gov.states.ma.tax.income.exemptions.dependent": {
    "2023-01-01.2100-12-31": 0
  },
  "gov.states.ms.tax.income.exemptions.dependents.amount": {
    "2023-01-01.2100-12-31": 0
  },
  "gov.states.sc.tax.income.deductions.young_child.amount": {
    "2023-01-01.2100-12-31": 0
  }
   }, country_id="us")


baseline = Microsimulation(dataset = "pooled_3_year_cps_2023")
reformed = Microsimulation(reform=reform, dataset = "pooled_3_year_cps_2023")
baseline_income = baseline.calculate("household_net_income", period=2024)
reformed_income = reformed.calculate("household_net_income", period=2024)
difference_income = reformed_income - baseline_income

# Calculate the baseline weight at the household level
baseline_weight = baseline.weights


In [ ]:
# Filter for non-zeros

difference_income = difference_income[difference_income != 0]

In [ ]:

# Include state code, difference, and weighted baseline in dataframe

state_codes = baseline.calculate("state_code", period=2024)


df = pd.DataFrame({
        'State': state_codes,
        'Difference': difference_income,
        'Weight' : baseline_weight
    })




In [ ]:
# Drop NA's

df = df.dropna()

,State,Difference
0,ME,-300.000000
2,ME,-300.000000
5,ME,-300.000000
14,ME,-300.000000
16,ME,-600.000000
...,...,...
54712,CA,-461.000000
54714,CA,-922.000000
54718,CA,-461.000000
54723,CA,-760.312500


In [ ]:
# Create a new column for weighted difference
df['Weighted_Difference'] = df['Difference'] * df['Weight']

# Difference in weighted income by state
total_difference = df.groupby('State')['Weighted_Difference'].sum() / 1e6

In [ ]:
total_difference

,Difference
State,
AL,-17681.406250
CA,-887980.812500
GA,-126659.132812
IA,-16241.019531
IL,-115635.039062
IN,-29139.203125
KS,-57603.703125
LA,-12626.316406
MA,-35747.109375


In [ ]:
# Including number of dependents

dependents = baseline.calculate("tax_unit_dependents", period=2024, map_to = "household")

In [ ]:
# Map state, difference, and dependents, to state_codes, difference_income, and dependents

df_new = pd.DataFrame({
        'State': state_codes,
        'Difference': difference_income,
        'Weight' : baseline_weight,
        'Number of Dependents': dependents
    })

df_new['Weighted_Difference'] = df_new['Difference'] * df_new['Weight']

In [ ]:
# Drop NA's

df_new = df_new.dropna()

df_new.head()

,State,Difference,Number of Dependents
0,ME,-300.000000,1.0
2,ME,-300.000000,1.0
5,ME,-300.000000,1.0
14,ME,-300.000000,1.0
16,ME,-600.000000,2.0
...,...,...,...
54712,CA,-461.000000,1.0
54714,CA,-922.000000,2.0
54718,CA,-461.000000,1.0
54723,CA,-760.312500,2.0


In [ ]:
from google.colab import files

# Download csv

df_new.to_csv('df_new.csv', index=False)
files.download('df_new.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>